# PROBLEM STATEMENT : 
#### URL : 

https://machinehack.com/hackathon/predict_accident_risk_score_for_unique_postcode/overview

According to IBEF “Domestic automobiles production increased at 2.36% CAGR between FY16-20 with 26.36 million vehicles being manufactured in the country in FY20.Overall, domestic automobiles sales increased at 1.29% CAGR between FY16-FY20 with 21.55 million vehicles being sold in FY20”.The rise in vehicles on the road will also lead to multiple challenges and the road will be more vulnerable to accidents.

Increased accident rates also leads to more insurance claims and payouts rise for insurance companies.

In order to pre-emptively plan for the losses, the insurance firms leverage accident data to understand the risk across the geographical units e.g. Postal code/district etc. In this challenge, we are providing you the dataset to predict the “Accident_Risk_Index” against the postcodes.

Accident_Risk_Index (mean casualties at a postcode) = sum(Number_of_casualities)/count(Accident_ID)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from scipy import stats

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
population=pd.read_csv("population.csv")
sample_sub=pd.read_csv("sample_submission.csv")
roads_network=pd.read_csv("roads_network.csv")

In [4]:
roads_network=roads_network.dropna()
roads_network=roads_network.reset_index(drop=True)
roads_network.isnull().sum().sum()

0

In [5]:
print('train : ', train.shape)
print('test : ', test.shape)
print('population : ', population.shape)
print('roads_network : ',roads_network.shape)

train :  (478741, 27)
test :  (121259, 27)
population :  (8035, 10)
roads_network :  (90352, 8)


In [6]:
print('train : ', train.postcode.nunique())
print('test : ', test.postcode.nunique())
print('population : ', population.postcode.nunique())
print('roads_network : ',roads_network.postcode.nunique())

train :  95625
test :  49772
population :  8035
roads_network :  74681


In [8]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Accident_ID,478741.0,299892.954673,173261.274950,1.0,149718.0,299791.0,449990.0,599999.0
Police_Force,478741.0,31.610178,24.941660,1.0,12.0,33.0,45.0,98.0
Number_of_Vehicles,478741.0,1.745695,0.626458,1.0,1.0,2.0,2.0,4.0
Number_of_Casualties,478741.0,1.463528,0.816019,1.0,1.0,1.0,2.0,5.0
Day_of_Week,478741.0,4.367898,1.872983,1.0,3.0,5.0,6.0,7.0
Local_Authority_(District),478741.0,357.457132,256.530105,1.0,103.0,314.0,521.0,941.0
1st_Road_Class,478741.0,4.381542,1.369579,1.0,3.0,4.0,6.0,6.0
1st_Road_Number,478741.0,946.663375,1730.926876,0.0,10.0,336.0,569.0,9832.0
Speed_limit,478741.0,37.035976,12.854299,20.0,30.0,30.0,40.0,70.0
2nd_Road_Class,478741.0,2.430068,3.268563,-1.0,-1.0,3.0,6.0,6.0


In [9]:
roads_network.head()

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode
0,POINT (-2.3501 56.603923),A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1
1,POINT (-2.021334 57.130142),A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN
2,POINT (-2.108598 57.146338),A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH
3,POINT (-2.093928 57.148218),A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL
4,POINT (-2.116089 57.131671),A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT


In [10]:
population.head()

,postcode,Rural Urban,Variable: All usual residents; measures: Value,Variable: Males; measures: Value,Variable: Females; measures: Value,Variable: Lives in a household; measures: Value,Variable: Lives in a communal establishment; measures: Value,Variable: Schoolchild or full-time student aged 4 and over at their non term-time address; measures: Value,Variable: Area (Hectares); measures: Value,Variable: Density (number of persons per hectare); measures: Value
0,AL1 1,Total,5453,2715,2738,5408,45,75,225.63,24.2
1,AL1 2,Total,6523,3183,3340,6418,105,77,286.59,22.8
2,AL1 3,Total,4179,2121,2058,4100,79,46,97.12,43.0
3,AL1 4,Total,9799,4845,4954,9765,34,285,244.75,40.0
4,AL1 5,Total,10226,5129,5097,10211,15,133,200.93,50.9


In [11]:
population.drop(['Rural Urban','Variable: Females; measures: Value'],axis=1,inplace=True)

In [12]:
population.columns=['postcode','residents','males','ll','CE','schoolchild','area','density']


In [13]:
population.males=population.males/population.residents
population.head()

,postcode,residents,males,ll,CE,schoolchild,area,density
0,AL1 1,5453,0.497891,5408,45,75,225.63,24.2
1,AL1 2,6523,0.487966,6418,105,77,286.59,22.8
2,AL1 3,4179,0.507538,4100,79,46,97.12,43.0
3,AL1 4,9799,0.494438,9765,34,285,244.75,40.0
4,AL1 5,10226,0.501565,10211,15,133,200.93,50.9


In [15]:
roads_network.head()

,WKT,roadClassi,roadFuncti,formOfWay,length,primaryRou,distance to the nearest point on rd,postcode,lat,log
0,-2.3501 56.603923,A Road,A Road,Single Carriageway,2643.0,1.0,1.256769,AB1,-2.350100,56.603923
1,-2.021334 57.130142,A Road,A Road,Single Carriageway,2643.0,1.0,1.834101,AB1 9NN,-2.021334,57.130142
2,-2.108598 57.146338,A Road,A Road,Single Carriageway,2643.0,1.0,1.830243,AB10 1UH,-2.108598,57.146338
3,-2.093928 57.148218,A Road,A Road,Single Carriageway,2643.0,1.0,1.835092,AB10 1YL,-2.093928,57.148218
4,-2.116089 57.131671,A Road,A Road,Single Carriageway,2643.0,1.0,1.814373,AB10 6AT,-2.116089,57.131671


In [14]:
roads_network.WKT=roads_network['WKT'].apply(lambda x:x.split('(')[1].split(')')[0])
roads_network['lat']=roads_network['WKT'].apply(lambda x:float(x.split(' ')[0]))
roads_network['log']=roads_network['WKT'].apply(lambda x:float(x.split(' ')[1]))

In [20]:
rn=roads_network[['lat','log','postcode']]
rn.head()

,lat,log,postcode
0,-2.350100,56.603923,AB1
1,-2.021334,57.130142,AB1 9NN
2,-2.108598,57.146338,AB10 1UH
3,-2.093928,57.148218,AB10 1YL
4,-2.116089,57.131671,AB10 6AT


# EDA

# Accident_Risk_Index (mean casualties at a postcode) = sum(Number_of_casualities)/count(Accident_ID)

In [21]:
sum_casualties = train.groupby([train['postcode']])['Number_of_Casualties'].sum()
count_acc = train.groupby([train['postcode']])['Accident_ID'].count()

Accident_Risk_Index = sum_casualties / count_acc
df  = pd.merge(train, Accident_Risk_Index.reset_index(name='Accident_Risk_Index'), how='left', on=['postcode'])

In [22]:
df.head()

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,...,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,Accident_Risk_Index
0,1,34,2,1,19/12/12,7,13:20,344,E10000032,4,...,Fine without high winds,Dry,Ol or diesel,None,1,Yes,England,OX3 9UP,United Kingdom,1.833333
1,2,5,2,1,02/11/12,4,7:53,102,E09000026,3,...,Raining without high winds,Dry,None,None,1,No,England,S35 4EZ,United Kingdom,1.333333
2,3,1,2,1,02/11/12,4,16:00,531,E10000016,6,...,Fine without high winds,Dry,None,None,1,No,England,BN21 2XR,United Kingdom,1.333333
3,4,1,1,1,06/05/12,1,16:50,7,E08000035,6,...,Fine without high winds,Dry,Roadworks,None,1,Yes,England,TA20 3PT,United Kingdom,1.666667
4,5,46,1,1,30/06/12,3,13:25,519,E10000031,3,...,Fine without high winds,Dry,None,None,1,No,England,DN20 0QF,United Kingdom,1.500000


# Concating train and test data into single Dataframe for preprocessing¶

In [23]:
df['train_or_test']='train'
test['train_or_test']='test'
df1=pd.concat([df,test])

# Missing value Imputation

In [24]:
df1['Time'].fillna(df1['Time'].mode().values[0],inplace=True)
df1['Road_Surface_Conditions'].fillna(df1['Road_Surface_Conditions'].mode().values[0],inplace=True)
df1['Special_Conditions_at_Site'].fillna(df1['Special_Conditions_at_Site'].mode().values[0],inplace=True)

In [25]:
cols_dropped = ['country','Local_Authority_(Highway)','Local_Authority_(District)','Accident_ID','1st_Road_Number',
                'Number_of_Casualties','2nd_Road_Number']

In [26]:
df1.drop(cols_dropped,axis=1,inplace=True)

In [27]:
df1['Time']=pd.to_datetime(df1['Time'], format='%H:%M').dt.hour

In [28]:
df1['Year'] = pd.to_datetime(df1['Date']).dt.year
df1['Month'] = pd.to_datetime(df1['Date']).dt.month
df1['Day'] = pd.to_datetime(df1['Date']).dt.day
df1['weekend']=df1['Day_of_Week'].apply(lambda x: 0 if x<6 else 1)

In [29]:
def time_of_day(time):
    hour = int(time)
    if hour<=6:
        return 0
    elif hour<=9:
        return 1
    elif hour<=5:
        return 2
    elif hour<21:
        return 3
    else:
        return 4

In [30]:
df1['Time']=df1['Time'].apply(time_of_day)

In [40]:
def county_code(postcode):
    postcode=postcode.split(' ')[0]
    only_alpha=''
    for char in postcode:
        if ord(char)>=65 and ord(char)<=90:
            only_alpha+=char
        elif ord(char)>=97 and ord(char)<=122:
            only_alpha+=char
    return only_alpha.upper()

In [31]:
df1.head(2)

,Police_Force,Number_of_Vehicles,Date,Day_of_Week,Time,1st_Road_Class,Road_Type,Speed_limit,2nd_Road_Class,Pedestrian_Crossing-Human_Control,...,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,Accident_Risk_Index,train_or_test,Year,Month,Day,weekend
0,34,2,19/12/12,7,3,4,Single carriageway,30,-1,None within 50 metres,...,1,Yes,England,OX3 9UP,1.833333,train,2012,12,19,1
1,5,2,02/11/12,4,1,3,One way street,30,-1,None within 50 metres,...,1,No,England,S35 4EZ,1.333333,train,2012,2,11,0


In [32]:
rn['postcode']=rn['postcode'].apply(lambda x : x[0:-2])
rn['postcode']=rn['postcode'].apply(lambda x : x.upper())
rn.rename(columns={'postcode':'postcode_merge'},inplace=True)

C:\Users\91844\AppData\Local\Temp/ipykernel_29388/394084880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rn['postcode']=rn['postcode'].apply(lambda x : x[0:-2])
C:\Users\91844\AppData\Local\Temp/ipykernel_29388/394084880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rn['postcode']=rn['postcode'].apply(lambda x : x.upper())
C:\Users\91844\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [34]:
rn=rn.groupby('postcode_merge',as_index=False).agg({'lat' : 'mean','log' : 'mean'})
rn

,postcode_merge,lat,log
0,AB,-2.704352,56.901920
1,AB1,-2.070907,56.809341
2,AB1 9,-2.021334,57.130142
3,AB10 1,-2.101263,57.147278
4,AB10 6,-2.118697,57.135466
...,...,...,...
10724,YO8 9,-1.107228,53.772852
10725,YO89 9,-1.075875,53.786824
10726,YO91 1,-1.079029,53.975455
10727,YS2,-0.557006,54.456617


In [36]:
df1['postcode_merge']=df1['postcode'].apply(lambda x : x[0:-2])
df1  = pd.merge(df1, rn, how='left',on=['postcode_merge'])
df1.head()

,Police_Force,Number_of_Vehicles,Date,Day_of_Week,Time,1st_Road_Class,Road_Type,Speed_limit,2nd_Road_Class,Pedestrian_Crossing-Human_Control,...,postcode,Accident_Risk_Index,train_or_test,Year,Month,Day,weekend,postcode_merge,lat,log
0,34,2,19/12/12,7,3,4,Single carriageway,30,-1,None within 50 metres,...,OX3 9UP,1.833333,train,2012,12,19,1,OX3 9,-1.235937,51.757932
1,5,2,02/11/12,4,1,3,One way street,30,-1,None within 50 metres,...,S35 4EZ,1.333333,train,2012,2,11,0,S35 4,-1.486937,53.477343
2,1,2,02/11/12,4,3,6,Roundabout,40,6,None within 50 metres,...,BN21 2XR,1.333333,train,2012,2,11,0,BN21 2,0.141196,50.852716
3,1,1,06/05/12,1,3,6,Single carriageway,30,6,None within 50 metres,...,TA20 3PT,1.666667,train,2012,6,5,0,TA20 3,-3.013006,50.880963
4,46,1,30/06/12,3,3,3,Dual carriageway,30,6,None within 50 metres,...,DN20 0QF,1.500000,train,2012,6,30,0,DN20 0,-0.502351,53.569844


In [46]:
df1[df1.train_or_test=='test'].isnull().sum()

Police_Force                                        0
Number_of_Vehicles                                  0
Date                                                0
Day_of_Week                                         0
Time                                                0
1st_Road_Class                                      0
Road_Type                                           0
Speed_limit                                         0
2nd_Road_Class                                      0
Pedestrian_Crossing-Human_Control                   0
Pedestrian_Crossing-Physical_Facilities             0
Light_Conditions                                    0
Weather_Conditions                                  0
Road_Surface_Conditions                             0
Special_Conditions_at_Site                          0
Carriageway_Hazards                                 0
Urban_or_Rural_Area                                 0
Did_Police_Officer_Attend_Scene_of_Accident         0
state                       

In [44]:
df1.isnull().sum()

Police_Force                                        0
Number_of_Vehicles                                  0
Date                                                0
Day_of_Week                                         0
Time                                                0
1st_Road_Class                                      0
Road_Type                                           0
Speed_limit                                         0
2nd_Road_Class                                      0
Pedestrian_Crossing-Human_Control                   0
Pedestrian_Crossing-Physical_Facilities             0
Light_Conditions                                    0
Weather_Conditions                                  0
Road_Surface_Conditions                             0
Special_Conditions_at_Site                          0
Carriageway_Hazards                                 0
Urban_or_Rural_Area                                 0
Did_Police_Officer_Attend_Scene_of_Accident         0
state                       

In [41]:
df1['county_code']=df1['postcode'].apply(county_code)

In [42]:
cols= ['lat', 'log']

for i in cols:
    df1[i] = df1[i].fillna(df1.groupby('county_code')[i].transform('mean'))

In [45]:
for i in cols:
    df1[i] = df1[i].fillna(df1.groupby('state')[i].transform('mean'))

In [52]:
df1.columns

Index(['Police_Force', 'Number_of_Vehicles', 'Day_of_Week', 'Time',
       '1st_Road_Class', 'Road_Type', 'Speed_limit', '2nd_Road_Class',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'state', 'Accident_Risk_Index', 'train_or_test',
       'Year', 'Month', 'Day', 'weekend', 'lat', 'log'],
      dtype='object')

In [57]:
df1.iloc[1]

Police_Force                          5
Number_of_Vehicles                    2
Day_of_Week                           4
Time                                  1
1st_Road_Class                        3
Road_Type                             1
Speed_limit                          30
2nd_Road_Class                       -1
Light_Conditions                      4
Weather_Conditions                    5
Road_Surface_Conditions               0
Special_Conditions_at_Site            3
Carriageway_Hazards                   3
Urban_or_Rural_Area                   1
state                                 2
Accident_Risk_Index            1.333333
train_or_test                     train
Year                               2012
Month                                 2
Day                                  11
weekend                               0
lat                           -1.486937
log                           53.477343
pc                                   12
lw                                   40


In [50]:
df1.drop(['postcode','county_code','postcode_merge','Date','Did_Police_Officer_Attend_Scene_of_Accident'],axis=1,inplace=True)

In [53]:
df1['pc']=df1['Pedestrian_Crossing-Human_Control']+df1['Pedestrian_Crossing-Physical_Facilities']
df1['lw']=df1['Light_Conditions']+df1['Weather_Conditions']
df1['rc']=df1['Road_Surface_Conditions']+df1['Special_Conditions_at_Site']+df1['Carriageway_Hazards']

In [55]:
df1.drop(['Pedestrian_Crossing-Human_Control','Pedestrian_Crossing-Physical_Facilities'],axis=1,inplace=True)

In [56]:
from sklearn.preprocessing import LabelEncoder
categ_col=['Road_Type', 'Light_Conditions','pc','lw','rc',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards','state']
for c in categ_col:
    le = LabelEncoder()
    df1[c] = le.fit_transform(df1[c])

In [59]:
df2=df1.drop(['Accident_Risk_Index','train_or_test'],axis=1)

In [64]:
df1[cols].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
lat,600000.0,-1.412200,1.356557,-6.306240,-2.317185,-1.401829,-0.231591,1.742991
log,600000.0,52.689252,1.439188,49.919436,51.510356,52.480112,53.583483,58.010929


In [65]:
df1["lat"] = pd.qcut(df1["lat"], 10, labels=False)
df1["log"] = pd.qcut(df1["log"], 10, labels=False)
#df1['zone'] = df1['lat'].astype(str)+'_'+df1['long'].astype(str)
#df1 = df1.drop(['lat','log'],axis=1)

In [67]:
df1.head(2)

,Police_Force,Number_of_Vehicles,Day_of_Week,Time,1st_Road_Class,Road_Type,Speed_limit,2nd_Road_Class,Light_Conditions,Weather_Conditions,...,train_or_test,Year,Month,Day,weekend,lat,log,pc,lw,rc
0,34,2,7,3,4,3,30,-1,4,1,...,train,2012,12,19,1,5,3,12,36,18
1,5,2,4,1,3,1,30,-1,4,5,...,train,2012,2,11,0,4,7,12,40,14


In [68]:
train1=df1[df1.train_or_test=='train']
test1=df1[df1.train_or_test=='test']
train1.drop('train_or_test',axis=1,inplace=True)
test1.drop('train_or_test',axis=1,inplace=True)

C:\Users\91844\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [69]:
target = train1['Accident_Risk_Index']

train_df = train1.drop(columns =['Accident_Risk_Index'])

test_df = test1.drop(columns=['Accident_Risk_Index'])

# MODEL

In [70]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

In [81]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

In [79]:
def objective(trial, data=train_df, target=target):
    X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.2, random_state=42)

    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 32),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_float('gamma', 0.0001, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 0.0001, 10.0, log = True),
        'lambda': trial.suggest_float('lambda', 0.0001, 10.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.8),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'booster': 'gblinear',
        'random_state': 42,
        'eval_metric': 'rmse'

    }
    
    model = xgb.XGBRegressor(**params)  
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 333, verbose = False)
    y_pred = model.predict(X_val)
    mse = metric(y_val, y_pred)

    return mse

In [83]:
study = optuna.create_study()
study.optimize(objective, n_trials = 50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

[I 2022-03-28 18:07:17,040] A new study created in memory with name: no-name-bb94be3d-f33c-47fa-a85d-946f9446f6b3


[18:07:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-03-28 18:07:26,471] Trial 0 finished with value: 0.3617102110255804 and parameters: {'max_depth': 16, 'learning_rate': 0.08, 'n_estimators': 5136, 'min_child_weight': 50, 'gamma': 0.0317364093363946, 'alpha': 3.6324148467840165, 'lambda': 0.01242773891568861, 'colsample_bytree': 0.37918945249870417, 'subsample': 0.4288030236098468}. Best is trial 0 with value: 0.3617102110255804.


[18:07:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-03-28 18:08:23,624] Trial 1 finished with value: 0.3616120023988419 and parameters: {'max_depth': 29, 'learning_rate': 0.08, 'n_estimators': 4607, 'min_child_weight': 67, 'gamma': 0.15571692151052208, 'alpha': 0.054242045578479545, 'lambda': 0.01693909522634645, 'colsample_bytree': 0.18840042224483478, 'subsample': 0.6235096295570068}. Best is trial 1 with value: 0.3616120023988419.


...

[I 2022-03-28 19:31:14,862] Trial 41 finished with value: 0.36122915224853436 and parameters: {'max_depth': 13, 'learning_rate': 0.08, 'n_estimators': 6873, 'min_child_weight': 238, 'gamma': 0.044164003522713914, 'alpha': 0.00023374861931800683, 'lambda': 0.001039073193770106, 'colsample_bytree': 0.5046604613098994, 'subsample': 0.5683634641053974}. Best is trial 17 with value: 0.3612260417821249.


[19:31:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-03-28 19:33:53,718] Trial 42 finished with value: 0.36122841244257003 and parameters: {'max_depth': 8, 'learning_rate': 0.08, 'n_estimators': 7017, 'min_child_weight': 160, 'gamma': 0.009871826831051008, 'alpha': 0.00020329970053225435, 'lambda': 0.001210583901347616, 'colsample_bytree': 0.4874780627615151, 'subsample': 0.5317568074915056}. Best is trial 17 with value: 0.3612260417821249.


[19:33:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




KeyboardInterrupt: 

In [72]:
def k_fold_cross_valid(model,x_train,y_train,n_splits=5):
    
    X = x_train.copy()
    y = y_train.copy()

    from sklearn.model_selection import KFold
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)
    res = []

    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
    
        res.append(metric(y_test,y_pred))
        
    print("RMSE:",np.array(res).mean())

In [71]:
def metric(y_test, y_pred):
    rmse = mean_squared_error(y_test, y_pred , squared=False)
    return rmse

In [87]:
%%time
xgb_reg = xgb.XGBRegressor(n_estimators=100,max_depth=6,
                                learning_rate=0.08,
                                subsample=0.7,gamma = 0.067,alpha=0.0004,
                                colsample_bytree=0.6,min_child_weight=240,
                                random_state = 42)
k_fold_cross_valid(xgb_reg,train_df,target,n_splits=3)

RMSE: 0.3598296035265301
Wall time: 1min 21s


In [88]:
xgb_pred = xgb_reg.predict(test_df)
xgbp = pd.DataFrame(xgb_pred,columns=['Accident_risk_index'])
final1=pd.concat([test['postcode'],xgbp['Accident_risk_index']],axis=1)
final1=final1.groupby('postcode',as_index=False).agg({'Accident_risk_index' : 'mean'})
final1.to_csv("xgb_pred.csv",index=False)

In [89]:
final1

,postcode,Accident_risk_index
0,AB10 1AU,1.455465
1,AB10 1PG,1.460550
2,AB10 1TT,1.503684
3,AB10 1YP,1.458755
4,AB10 6LQ,1.425075
...,...,...
49767,ZE2 9LZ,1.429066
49768,ZE2 9RE,1.435767
49769,ZE2 9RJ,1.412882
49770,ZE2 9SB,1.486296
